In [13]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecMonitor, VecFrameStack, DummyVecEnv

from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CheckpointCallback
import torch

def make_procgen_env(env_name, num_envs=1, start_level=0, num_levels=0):
    def _init():
        
        return env
    return DummyVecEnv([_init] * num_envs)

env_name = "procgen:procgen-heist-v0"  
num_envs = 4 
learning_rate = 3e-4
n_steps = 2048
batch_size = 64
n_epochs = 10
gamma = 0.99

env = gym.make(env_name, start_level=0, num_levels=1, render_mode="rgb_array")
# env = make_procgen_env(env_name, num_envs)






In [14]:
import sys
sys.path.append('../') 

from src.policies import ImpalaCNN

observation_space = env.observation_space

action_space = env.action_space.n

In [15]:
def load_pretrained_model(model_path):
    model = ImpalaCNN(observation_space, action_space)
    model_state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.state_dict = model_state_dict
    model.eval()  
    return model

model_path = '../heist_model_final_easy.pt'
model = load_pretrained_model(model_path)



In [19]:

def generate_action(model, observation, device='cpu'):
    observation = torch.tensor(observation, dtype=torch.float32).unsqueeze(0).to(device)
    logits = model(observation)
    print(logits)
    probabilities = torch.softmax(logits, dim=-1)
    action = probabilities.multinomial(num_samples=1).item()  # For single observation
    return action



In [20]:
env.action_space.sample()

8

In [21]:
observation = env.reset()
done = False

action = env.action_space.sample()


counter = 0 
while not done:
    print(counter)
    # Generate a random action
    
    # Apply the action to the environment
    observation, reward, done, info = env.step(action)
    action = generate_action(model, observation=observation)

    
    # Optionaly render the environment
    env.render()
    

0
(Categorical(logits: torch.Size([1, 15])), tensor([[0.0004]], grad_fn=<AddmmBackward0>))


TypeError: softmax() received an invalid combination of arguments - got (tuple, dim=int), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype, *, Tensor out)
 * (Tensor input, name dim, *, torch.dtype dtype)
